In [1]:
from llama_index.core import StorageContext, load_index_from_storage

from constants import embed_model

storage_context = StorageContext.from_defaults(persist_dir="index/")
index = load_index_from_storage(storage_context, embed_model=embed_model)

In [2]:
from llama_index.core.tools import QueryEngineTool

from constants import llm_model

query_engine = index.as_query_engine(llm_model=llm_model, similarity_top_k=5)
rag_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="research_paper_query_engine_tool",
    description="A RAG engine with recent research papers.",
)

In [3]:
from IPython.display import Markdown, display


def display_prompt_dict(prompts_dict):
    for key, prompt in prompts_dict.items():
        display(Markdown(f"**Prompt key**: {key}"))
        print(prompt.get_template())


In [4]:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt key**: response_synthesizer:text_qa_template

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt key**: response_synthesizer:refine_template

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [5]:
from llama_index.core.tools import FunctionTool

from tools import download_pdf, fetch_arxiv_papers

download_pdf_tool = FunctionTool.from_defaults(
    download_pdf,
    name="download_pdf_file_tool",
    description="python function that downloads a pdf file by link",
)

fetch_arxiv_tool = FunctionTool.from_defaults(
    fetch_arxiv_papers,
    name="fetch_from_arxiv",
    description="download the {max_results} recent papers regarding the topic {title} from arXiv",
)

In [6]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    [download_pdf_tool, rag_tool, fetch_arxiv_tool], llm=llm_model, verbose=True
)

In [7]:
query_template = """I am interested in {topic}.
Find papers in your knowledge database related to this topic.
Use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to {topic}'.
If there are not, could you fetch the recent one from arXiv?
"""

In [8]:
answer = agent.chat(query_template.format(topic="Multi-Modal Models"))

> Running step dfe2c681-b09d-4039-85fe-0f6d2f98dd6b. Step input: I am interested in Multi-Modal Models.
Find papers in your knowledge database related to this topic.
Use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to Multi-Modal Models'.
If there are not, could you fetch the recent one from arXiv?

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: research_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, authors and link to download for papers related to Multi-Modal Models'}
Observation: Title: Survey of Large Multimodal Model Datasets, Application Categories and Taxonomy  
Summary: Multimodal learning, a rapidly evolving field in artificial intelligence, seeks to construct more versatile and robust systems by integrating and analyzing diverse types of data, including text, images, audio, and video.

In [9]:
Markdown(answer.response)

Here are some recent papers related to Multi-Modal Models:

1. **Title:** Survey of Large Multimodal Model Datasets, Application Categories and Taxonomy  
   **Summary:** Multimodal learning, a rapidly evolving field in artificial intelligence, seeks to construct more versatile and robust systems by integrating and analyzing diverse types of data, including text, images, audio, and video.  
   **Authors:** Priyaranjan Pattnayak, Hitesh Laxmichand Patel, Bhargava Kumar, Amit Agarwal, Ishan Banerjee, Srikant Panda, Tejaswini Kumar  
   **PDF URL:** [Download Here](http://arxiv.org/pdf/2412.17759v1)

2. **Title:** Cross-Lingual Text-Rich Visual Comprehension: An Information Theory Perspective  
   **Summary:** Recent Large Vision-Language Models (LVLMs) have shown promising reasoning capabilities on text-rich images from charts, tables, and documents.  
   **Authors:** Xinmiao Yu, Xiaocheng Feng, Yun Li, Minghui Liao, Ya-Qi Yu, Xiachong Feng, Weihong Zhong, Ruihan Chen, Mengkang Hu, Jihao Wu, Dandan Tu, Duyu Tang, Bing Qin  
   **PDF URL:** [Download Here](http://arxiv.org/pdf/2412.17787v1)

3. **Title:** Comprehensive Multi-Modal Prototypes are Simple and Effective Classifiers for Vast-Vocabulary Object Detection  
   **Summary:** Enabling models to recognize vast open-world categories has been a longstanding pursuit in object detection.  
   **Authors:** Yitong Chen, Wenhao Yao, Lingchen Meng, Sihong Wu, Zuxuan Wu, Yu-Gang Jiang  
   **PDF URL:** [Download Here](http://arxiv.org/pdf/2412.17800v1)

4. **Title:** ChatGarment: Garment Estimation, Generation and Editing via Large Language Models  
   **Summary:** We introduce ChatGarment, a novel approach that leverages large vision-language models (VLMs) to automate the estimation, generation, and editing of 3D garments from images or text descriptions.  
   **Authors:** Siyuan Bian, Chenghao Xu, Yuliang Xiu, Artur Grigorev, Zhen Liu, Cewu Lu, Michael J. Black, Yao Feng  
   **PDF URL:** [Download Here](http://arxiv.org/pdf/2412.17811v1)

In [10]:
answer = agent.chat("Download all the papers you mentioned.")

> Running step b5b7fa00-1cba-47e3-ac6c-3e4ee24429b8. Step input: Download all the papers you mentioned.
Thought: I need to download multiple PDF files based on the provided URLs for the papers related to Multi-Modal Models.
Action: download_pdf_file_tool
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2412.17759v1', 'output_file_name': 'Survey_of_Large_Multimodal_Model_Datasets.pdf'}
Observation: PDF downloaded successfully and saved as 'papers/Survey_of_Large_Multimodal_Model_Datasets.pdf'.
> Running step dd3b2974-9056-4688-bdf3-080e295d4018. Step input: None
Thought: (Implicit) I can answer without any more tools!
Answer: Action: download_pdf_file_tool  
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2412.17787v1', 'output_file_name': 'Cross-Lingual_Text-Rich_Visual_Comprehension.pdf'}


In [11]:
Markdown(answer.response)

Action: download_pdf_file_tool  
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2412.17787v1', 'output_file_name': 'Cross-Lingual_Text-Rich_Visual_Comprehension.pdf'}

In [12]:
answer = agent.chat(query_template.format(topic="Quantum Computing"))

> Running step 47e908a4-3d25-43d4-8501-bee35aa3a611. Step input: I am interested in Quantum Computing.
Find papers in your knowledge database related to this topic.
Use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to Quantum Computing'.
If there are not, could you fetch the recent one from arXiv?

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: research_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, authors and link to download for papers related to Quantum Computing'}
Observation: Title: None
Summary: None
Authors: None
PDF URL: None
arXiv URL: None
> Running step 4a3410e3-9c53-44dc-9911-5444610f323d. Step input: None
Thought: It seems there are no papers available in the knowledge database related to Quantum Computing. I will fetch recent papers from arXiv.
Action: fetch_from_arxiv
Action Input

In [13]:
Markdown(answer.response)

Here are some recent papers related to Quantum Computing:

1. **Title:** Probing Entanglement Scaling Across a Quantum Phase Transition on a Quantum Computer  
   **Summary:** This paper addresses challenges in investigating strongly-correlated quantum matter, particularly near continuous quantum phase transitions. It utilizes the multiscale entanglement renormalization ansatz (MERA) on a trapped-ion quantum computer to probe many-body entanglement and demonstrates log-law scaling of subsystem entanglement entropies at criticality.  
   **Authors:** Qiang Miao, Tianyi Wang, Kenneth R. Brown, Thomas Barthel, Marko Cetina  
   **PDF URL:** [Download Here](http://arxiv.org/pdf/2412.18602v1)

2. **Title:** Uniform Additivity of Tripartite Optimized Correlation Measures  
   **Summary:** This paper explores optimized linear entropic functions of quantum states that are additive. It identifies convex polyhedral cones of uniformly additive tripartite correlation measures and proves the additivity of three previously established measures.  
   **Authors:** Joshua Levin, Ariel Shlosberg, Vikesh Siddhu, Graeme Smith  
   **PDF URL:** [Download Here](http://arxiv.org/pdf/2412.18586v1)

3. **Title:** Scalable Quantum-Inspired Optimization through Dynamic Qubit Compression  
   **Summary:** The paper presents a quantum-inspired framework that dynamically compresses large Ising models to fit available quantum hardware. It leverages a physics-inspired GNN architecture to capture interactions in Ising models and reduce model size while preserving optimization structure.  
   **Authors:** Co Tran, Quoc-Bao Tran, Hy Truong Son, Thang N Dinh  
   **PDF URL:** [Download Here](http://arxiv.org/pdf/2412.18571v1)

4. **Title:** A Time Optimization Framework for the Implementation of Robust and Low-latency Quantum Circuits  
   **Summary:** This paper introduces a pulse scheduling approach that enables the use of both fast and robust quantum gates within the same quantum circuit, improving the success probability of quantum circuit execution by over 25%.  
   **Authors:** Eduardo Willwock Lussi, Rafael de Santiago, Eduardo Inacio Duzzioni  
   **PDF URL:** [Download Here](http://arxiv.org/pdf/2412.18533v1)

5. **Title:** Hardware-aware Circuit Cutting and Distributed Qubit Mapping for Connected Quantum Systems  
   **Summary:** This project introduces DisMap, a hardware-aware framework for chip-to-chip distributed quantum systems, which analyzes qubit noise and error rates to enhance fidelity and reduce SWAP overhead in distributed quantum computing.  
   **Authors:** Zefan Du, Yanni Li, Zijian Mo, Wenqi Wei, Juntao Chen, Rajkumar Buyya, Ying Mao  
   **PDF URL:** [Download Here](http://arxiv.org/pdf/2412.18458v1)